Wig-Wag Frequency Visualizer Python Code

This is a code that will help visualization of frequency parameters. It will output a 10 second animation of desired parameters.


This code will allow the user to adjust parameters for the wig wag headlights including:

Flash (hz): the rate that it will cycle left off, right on -> right on left off

Hazard Flash (Hz): the rate at which the hazards light flash (NOTE: Currently set to 1.25hz)


How to use:

1: Click the double play arrows in the tool bar

2: Wait for the first image to appear

3: Adjust to desired frequency

4: Click Generate Animation and Wait

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, HTML
import io
import base64
import ipywidgets as widgets
from ipywidgets import FloatSlider, Button, Output

# Global variables to manage state
hz = 1.0  # Initial frequency for headlights
hazard_hz = 1.25  # Initial frequency for hazard lights

# Output widget to capture button actions
out = Output()

def create_animation(hz, hazard_hz):
    # Calculate on and off times based on frequency
    headlight_on_time = 1 / (2 * hz)
    headlight_off_time = 1 / (2 * hz)
    hazard_on_time = 1 / (2 * hazard_hz)
    hazard_off_time = 1 / (2 * hazard_hz)

    # Create figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 2)
    ax.axis('off')  # Hide axes
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    # Create a label for the frequencies
    frequency_label = f"Headlight Frequency: {hz:.2f} Hz\nHazard Frequency: {hazard_hz:.2f} Hz"
    ax.text(0.1, 1.5, frequency_label, color='white', fontsize=12, va='top')

    # Create two white circles for headlights
    headlight1 = plt.Polygon([(1.5, 0.75), (3, 0.75), (3.5, 0.5), (2, 0.5)], color='white', fill=True)
    headlight2 = plt.Polygon([(8.5, 0.75), (7, 0.75), (6.5, 0.5), (8, 0.5)], color='white', fill=True)
    ax.add_patch(headlight1)
    ax.add_patch(headlight2)

    # Create two white circles for hazard lights
    hazard1 = plt.Polygon([(1.25, 0.75), (1.5, 0.75), (2, 0.5), (1.75, 0.5)], color='orange', fill=True)
    hazard2 = plt.Polygon([(8.75, 0.75), (8.5, 0.75), (8, 0.5), (8.25, 0.5)], color='orange', fill=True)
    ax.add_patch(hazard1)
    ax.add_patch(hazard2)

    total_time = 10  # Fixed duration of the animation in seconds
    fps = 30         # Frames per second for the animation

    # Prepare to save animation frames
    frames = []
    for frame in range(int(total_time * fps)):
        cycle_time = (frame / fps) % (2 * (headlight_on_time + headlight_off_time))
        hazard_cycle_time = (frame / fps) % ((hazard_on_time + hazard_off_time))
        
        # Headlight animation
        if cycle_time < headlight_on_time:
            headlight1.set_visible(True)
            headlight2.set_visible(False)
        elif cycle_time < headlight_on_time + headlight_off_time:
            headlight1.set_visible(False)
            headlight2.set_visible(True)
        elif cycle_time < 2 * headlight_on_time + headlight_off_time:
            headlight1.set_visible(True)
            headlight2.set_visible(False)
        else:
            headlight1.set_visible(False)
            headlight2.set_visible(True)

        # Hazard light animation (flashing together)
        if hazard_cycle_time < hazard_on_time:
            hazard1.set_visible(True)
            hazard2.set_visible(True)
        else:
            hazard1.set_visible(False)
            hazard2.set_visible(False)

        # Capture the frame
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
        buf.seek(0)
        img = Image.open(buf)
        frames.append(img.copy())  # Ensure a copy of the image is added to the list
        buf.close()

    # Save all frames as a GIF
    gif_buffer = io.BytesIO()
    frames[0].save(
        gif_buffer,
        format='GIF',
        append_images=frames[1:],
        save_all=True,
        duration=1000 / fps,  # Duration per frame in milliseconds
        loop=0
    )
    gif_buffer.seek(0)

    # Encode GIF as base64
    gif_base64 = base64.b64encode(gif_buffer.read()).decode('utf-8')

    # Display GIF inline
    with out:
        out.clear_output()
        display(HTML(f'<img src="data:image/gif;base64,{gif_base64}" />'))
    plt.close(fig)  # Close the figure to avoid displaying it twice

def on_button_click(b):
    create_animation(hz, hazard_hz)  # Update animation with current frequencies

def update_headlight_frequency(change):
    global hz
    hz = change['new']  # Update the frequency value without generating animation

def update_hazard_frequency(change):
    global hazard_hz
    hazard_hz = change['new']  # Update the hazard frequency value

# Create interactive widgets
hz_slider = FloatSlider(value=1.0, min=0.1, max=2.0, step=0.01, description='Wig-Wag Flash (Hz):')
hazard_slider = FloatSlider(value=1.25, min=0.1, max=2.0, step=0.01, description='Hazard Flash (Hz):')
button = Button(description="Generate Animation")

# Register button action
button.on_click(on_button_click)

# Display widgets
display(hz_slider, hazard_slider, button, out)

# Link sliders to update frequency values
hz_slider.observe(update_headlight_frequency, names='value')
hazard_slider.observe(update_hazard_frequency, names='value')

# Create initial animation
create_animation(hz, hazard_hz)


FloatSlider(value=1.0, description='Wig-Wag Flash (Hz):', max=2.0, min=0.1, step=0.01)

FloatSlider(value=1.25, description='Hazard Flash (Hz):', max=2.0, min=0.1, step=0.01)

Button(description='Generate Animation', style=ButtonStyle())

Output()